# Training a character language model and studying various ways of generating text

**Author: matthieu.labeau@telecom-paris.fr**

## Objectives:

- We will train a network to predict a next character given an input sequence of characters, and use it to generate new sequences.
- We will strictly work with local (and not structured) prediction - however, we will look into relatively simple heuristics to improve the "structure" of our generation: *temperature* sampling, *beam search*, *top-k* sampling, *top-p* sampling, drop-out input data.
- We will use ```keras```to build the model based on a LSTM, which will use simple features (one-hot vector representing previous characters) to predict the next characters. We will use a small model to avoid training for too long.
- We will use a small dataset (poetry, from project Gutenberg) - you can use any data you prefer, as long as you are able to train the model on it.
- Even with a small dataset and a small model, training may be long (~1 min/epoch on GPU). If you can use a computing infrastructure, like Google colab, it may be more practical - and you probably can obtain better results by using a bigger model and a larger dataset.  

In [ ]:
import numpy as np
import random
import sys
import matplotlib.pyplot as plt

### Obtaining the data
- We download directly the ebook from project Gutenberg - you can get any other text you would prefer.

In [ ]:
from keras.utils import get_file
url = 'https://www.gutenberg.org/cache/epub/100/pg100.txt'
path = get_file('pg100.txt', origin=url)

f = open(path, 'r' , encoding = 'utf8')
lines = f.readlines()
text = []

start = False
for line in lines:
    line = line.strip().lower() # Remove blanks and capitals
    if("*** START OF THE PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***".lower() in line and start==False):
        start = True
    if("*** END OF THE PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***".lower() in line):
        break
    if(start==False or len(line) == 0):
        continue
    text.append(line)

f.close()
text = " ".join(text)
voc_chars = sorted(set([c for c in text]))
nb_chars = len(voc_chars)

In [ ]:
print(lines[0])

### Keeping track of possible characters
- Using a ```set```, create a sorted list of possible characters
- Create two dictionnaries, having characters and corresponding indexes as {key: value}, and reverse.

Example:

```python
chars = [a, b, c]
```

```python
chars_indices = {a: 0, b: 1, c: 2}
```

```python
indices_chars = {0: a, 1: b, 2: c}
```

In [ ]:
print('Corpus length:', len(text))

chars = ...
print('Total number of characters:', len(chars))
char_indices = ...
indices_char = ...

#### Parenthesis: looking at words

- Try to look at tokenization schemes: what are the most frequent words if we use *whitespace* tokenization ?
- What are the most frequent words in this dataset using smarter tokenization ? You can a tokenizer from NLTK: ```nltk.word tokenize```.
- Make a plot of rank vs. word count. Does Zipf's Law seem to hold ?


In [ ]:
from collections import Counter
# Apply whitespace tokenization
tokens = ...

# Get frequencies and sort words according to them
freq = ...
ordered_word_list = ...
print(ordered_word_list[:100])

del tokens
del freq
del ordered_word_list

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk import word_tokenize
tokens = ...

freq = ...
ordered_word_list = ...
print(ordered_word_list[:20])

In [ ]:
# Create an array containing the word rank in the first dimension, and its count in the second.
rank_counts = ...

In [ ]:
plt.figure(figsize=(20,5))
plt.title('Word counts versus rank')
plt.scatter(rank_counts[:,0], rank_counts[:,1])
plt.yscale('log')
plt.show()

print('Vocabulary size: %i' % len(freq))
print('Part of the corpus by taking the "x" most frequent words:')
for i in range(1000, len(freq), 1000):
    print('%i : %.2f' % (i, np.sum(rank_counts[:i, 1]) / np.sum(rank_counts[:,1]) ))

del tokens
del freq
del ordered_word_list
del rank_counts

### Creating a model: a first, simple version

This model will take as input **a fixed number of characters** and output the next one. We will treat the model as a *black box* and leave its innner working for later.

#### Creating training data
- We will represent characters using *one-hot vectors*. Hence, the i-th character of n possible characters will be represented by a vector of length $n$, containing $0$ expect for a $1$ in position $i$. Following our previous examples, ```a = [1, 0, 0]``` and ```b = [0, 1, 0]```.
- Hence, a sequence of characters is a list of one-hot vectors. Our goal will be to predict, given an input sequence of fixed length (here, this length is given by ```maximum_seq_length```) the next character. Hence, we need to build two lists: ```sentences```, containing the input sequences, and ```next_char``` the characters to be predicted.
- We do not necessarily need to take all possible sequences. We can select one every ```time_step``` steps.

Example: Using the previous dictionnaries, the sequence:
```'acabbaccaabba'``` with ```maximum_seq_length = 4``` and ```time_step = 2``` would give the following lists:

```python
sentences = ['acab', 'abba', 'bacc', 'ccaa', 'aabb']
```

```python
next_char = ['b', 'c', 'a', 'b', 'a']
```

In [ ]:
maximum_seq_length = 30
time_step = 4
sentences = []
next_char = []
for i in range(...):
    sentences.append(...)
    next_char.append(...)
print('Number of Sequences:', len(sentences))

#### Creating training tensors
- We need to transform these lists into tensors, using one-hot vectors to represent characters.
- We will need 3 dimensions for the training examples from ```sentences```: the number of examples, the length of the sequence, and the dimension of the one-hot vector
- This is reduced to 2 dimensions for the ```next_char```: number of examples and one-hot vector.

Example: the previous ```sentences``` would become:

```python
X = [[[1, 0, 0],
      [0, 0, 1],
      [1, 0, 0],
      [0, 1, 0]],
     [[1, 0, 0],
      [0, 1, 0],
      [0, 1, 0],
      [1, 0, 0]],
     [[0, 1, 0],
      [1, 0, 0],
      [0, 0, 1],
      [0, 0, 1]],
     [[0, 0, 1],
      [0, 0, 1],
      [1, 0, 0],
      [1, 0, 0]],
     [[1, 0, 0],
      [1, 0, 0],
      [0, 1, 0],
      [0, 1, 0]]]
```
       
```python
y = [[0, 1, 0],
     [0, 0, 1],
     [1, 0, 0],
     [0, 1, 0],
     [1, 0, 0]]
```

In [ ]:
X = np.zeros((len(sentences), maximum_seq_length, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
# Loop over the sentences
for ...
    # Loop over the characters
    for ...
        # Put the right value of X to 1
        ...
    # Put the right value of y to 1
    ...

#### Implement the model

In order to implement the model as simply as possible, we will use ```keras```. It allows to create models with only a few lines of code.
First, we will create a very simple model based on a **LSTM**, which is a *recurrent* architecture. Note that one the strength of a recurrent architecture is to allow for inputs of varying length - here, to simplify data processing, we will keep a **fixed input size**.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding
from keras.layers import LSTM
from keras.callbacks import LambdaCallback, EarlyStopping
from keras.metrics import sparse_categorical_crossentropy

We need to create a LSTM model that takes directly out inputs from ```X``` and try to predict one-hot vectors from ```y```.
- What are the input and output dimensions ?
  - ```X```: size of the dataset $\times$ maximum sequence length $\times$ vocabulary size
  - ```y```: size of the dataset $\times$ vocabulary size
- The model should be made with a ```LSTM``` layer, and a ```Dense``` layer followed by a softmax activation function. Work out the intermediate dimensions:
  - ```X``` $\rightarrow$ (LSTM) $\rightarrow$ ```h``` $\rightarrow$ (Dense) $\rightarrow$ ```s``` $\rightarrow$ (softmax) $\rightarrow$ ```pred```
  - Look at layers arguments and find out to proper ```input_shape``` for the ```LSTM``` layer and the proper size for the ```Dense``` layer.
  - You can use 128 as the size of hidden states for the ```LSTM```.
- We will minimize ```cross-entropy(pred, y)```. Use the ```categorical_crossentropy``` loss, with the optimizer of your preference (for example, ```adam```).

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(maximum_seq_length, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

You will now only need a few functions to use this model:
- ```model.fit```, which you will call on the appropriately processed data ```X, y```
- ```model.predict```, which you will use on an input **of the same dimension of X** to output the probabilities. That includes the *first one*, corresponding to the number of examples in the input.

### Create functions to generate text with our model
- We use the output of our model (vector of probabilities on characters) to select the next most probable character (with the ```argmax```)
- We need to transform an input text into an input tensor, as before (taking the right length, the last ```maximum_seq_length``` characters)
- We need to transform back the most probable index into a character and add it to our text.
- This must be looped ```num_generated``` times, each time obtaining a new input tensor from the new input sequence (which has the character we previously predicted at the end !)

We can begin to write a function facilitating the transfer between text and tensors:

In [ ]:
def get_tensor(sentence, maximum_seq_length, voc):
    x = np.zeros(...)
    # Fill out x appropriately
    ...
    return x

You have now what is necessary to fill out ```generate_next```.

The following function (```end_epoch_generate```) is here to facilitate automatic generation at the end of each epoch, so you can monitor of generation changes as the model trains. It calls the ```generate_next``` function upon each sequence of text in ```texts_ex```. The only element in this list right now comes from the training data - you can add your own. We also use ```EarlyStopping``` to stop training when validation loss does not decrease.

In [ ]:
def generate_next(model, text, num_generated=120):
    generated = text
    sentence = text[-maximum_seq_length:]
    for i in range(num_generated):
        # Obtain the representation for the sentence, the prediction, the index of the better character and the character itself.
        x = ...
        predictions = ...
        next_index = ...
        next_char = ...
        generated += next_char
        sentence = sentence[1:] + next_char
    return(generated)

def end_epoch_generate(epoch, _):
    print(' Generating text after epoch: %d' % (epoch+1))
    texts_ex = ["From fairest creatures we desire increase,"]
    for text in texts_ex:
        sample = generate_next(model, text.lower())
        print('%s' % (sample))

early_stopping = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=2,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True
)

Test generation with the model not yet trained:

In [ ]:
text_ex = "From fairest creatures we desire increase,"
generate_next(model, text_ex.lower())

In [ ]:
model.fit(X, y,
          batch_size=128,
          epochs=5,
          validation_split = 0.2,
          callbacks=[LambdaCallback(on_epoch_end=end_epoch_generate), early_stopping])

#### Sampling with our model
- Now, instead of simply selecting the most probable next character, we would like to be able to draw a sample from the distribution output by the model.
- To better control the generation, we would like to use the argument ```temperature```, to smooth the distribution.
- Use the ```multinomial``` function from the ```random``` package to draw samples.
- Integrate this into a function ```generate_sample``` that be almost exactly like ```generate_next```.

In [ ]:
def sample(predictions, temperature, placeholder=None):
    # From the prediction, apply the temperature to reweights the probabilities and sample.
    ...
    probas = ...
    return ...

def generate_sample(model, text, sample_function, *sampling_arg, num_generated=120, temperature=1.0):
    generated = text
    sentence = text[-maximum_seq_length:]
    for i in range(num_generated):
        x = ...
        predictions = ...
        next_index = sample_function(predictions, temperature, *sampling_arg)
        next_char = ...
        generated += next_char
        sentence = sentence[1:] + next_char
    return(generated)

In [ ]:
generate_sample(model, text_ex.lower(), sample, temperature = 0.7)

#### Generate text with the beam algorithm
- Complete this function implementing the beam procedure.
- We need to loop for each character we want to generate, keeping track of the best ```beam_size``` sequences at the most.
- Besides keeping track of past generated character for each of these ```beam_size``` sequences, we need to keep track of their log-probability.
- This is done by, at each loop, keeping the ```beam_size```best predictions for each of the ```beam_size``` sequences, computing the log-probabilities of the newly formed (```beam_size```)$^2$ , and keeping the overall ```beam_size``` best new sequences.

In [ ]:
def generate_beam(model, text, beam_size=5, num_generated=120):
    generated = text
    sentence = text[-maximum_seq_length:]
    # Initialization of the beam with log-probabilities for the sequence
    current_beam = [(0, [], sentence)]

    for l in range(num_generated):
        all_beams = []
        for prob, current_preds, current_input in current_beam:
            x = ...
            prediction = ...
            # beam_size best predictions !
            possible_next_chars = ...
            # Add to the beams: (the probability of the sequence, the sequence of indexes generated, the full sequence of characters)
            all_beams += [
                (...,
                 ...,
                 ...
                )
                for ... in possible_next_chars]

        # Sort the beams according to their probability and keep the beam_size best
        current_beam = ...

    return ...

In [ ]:
generate_beam(model, text_ex.lower())

#### Generate text with top-k sampling
- This is very similar to the previously implemented sampling procedure, but we would like to choose a parameter ```k```, which is used to limit the sampling to the top-$k$ results of the model prediction.

In [ ]:
def sample_top_k(predictions, temperature, k):
    ...
    indices_to_remove = ...
    predictions[indices_to_remove] = -float('Inf')
    ...

    return ...

In [ ]:
generate_sample(model, text_ex.lower(), sample_top_k, 5, temperature = 0.7)

#### Generate text with top-p sampling
- This is very similar to the previously implemented sampling procedure, but we would like to choose a parameter ```p```, which is used to limit the sampling to the top results of the model prediction corresponding together to at least the probability $p < 1$.

In [ ]:
def sample_top_p(predictions, temperature, p):
    ...

    cum_prob = 0.0
    incr = 0
    # Get indices in increasing order of probability
    indices = ...
    probs = predictions[indices]
    # Increment cum_prob with until it gets above 1 - p
    # Keep track of the indices, which will be those to remove
    while ...
      ...

    indices_to_remove = indices[incr:]
    log_predictions[indices_to_remove] = -float('Inf')
    ...
    return ...

In [ ]:
generate_sample(model, text_ex.lower(), sample_top_p, 0.75, temperature = 0.7)

#### Reranking
If we would like to use our outputs for a task, a common strategy is to generate a large number of them, and select the one that maximizes the relevant metric: this is **re-ranking**.

First, we need an evaluation measure. For now, we can use **perplexity**: given how cross-entropy is computed in ```keras```, perplexity is simply *the exponential of the mean cross-entropy over the sequence*.

To compute the cross-entropy, you need both the true ```y``` and the corresponding ```pred``` output by the model. While the most efficient would be to gather these while sampling, we can re-compute it for any sentence with the following function:

In [ ]:
def get_preds(sentence):
    # Reconstitute the inputs and outputs from the sentence: first as list of characters
    eval_input = []
    eval_target = []
    for i in range(0, len(sentence) - maximum_seq_length):
        eval_input.append(sentence[i: i + maximum_seq_length])
        eval_target.append(sentence[i + maximum_seq_length])

    # Then as tensors
    eval_X = np.zeros(...)
    for ...
        for ...
            ...
    ...

    eval_pred = ...
    return eval_y, eval_pred

In [ ]:
def perplexity(y_true, y_pred):
    cross_entropy = ...
    perplexity = ...
    return perplexity

In [ ]:
def re_rank(sentence):
    eval_y, eval_pred = get_preds(sentence)
    return perplexity(eval_y, eval_pred)

You can then generate a bunch of sentences, obtain their perplexity and sort them accordingly:

In [ ]:
reranked = dict()
for k in np.arange(5,10):
    for t in np.arange(0.5, 1.0, 0.05):
        sentence = generate_sample(model, text_ex.lower(), sample_top_k, k, temperature = 0.7)
        p = re_rank(sentence)
        reranked[sentence] = p

In [ ]:
print(sorted(reranked, key=reranked.get))

In [ ]:
print(reranked)

In [ ]:
del X
del y
del model

### Create a second model: with embeddings

This model will have a key difference with the previous one: we will use **dense embeddings** instead of one-hot representations.
This means that:
- In ```X```, we will use the index of the character instead of an one-hot vector. For example, the following ```['acab', 'abba', 'bacc', 'ccaa', 'aabb']``` will be represented as ```[[0, 2, 0, 1], [0, 1, 1, 0], [1, 0, 2, 2], [2, 2, 0, 0]]```.
- We will need another input layer before the LSTM. Instead of ```Dense```, keras has a dedicated layer: ```Embedding```.

Create the dataset, adapt the generation function, create the new model, and compare the results.

In [ ]:
X_emb = np.zeros(...)
y_emb = np.zeros(...)
for ...
    for ...
        ...
    ...

print('X_emb shape:', X_emb.shape)
print('y_emb shape:', y_emb.shape)

In [ ]:
from keras.layers import Embedding

In [ ]:
def generate_next_emb(model, text, num_generated=120):
    generated = text
    sentence = text[-maximum_seq_length:]
    char_idxs = [[char_indices[char] for char in sentence]]
    for i in range(num_generated):
        x = ...
        predictions = ...
        next_index = ...
        next_char = ...
        generated += next_char
        char_idxs = [char_idxs[0][1:] + [next_index]]
    return(generated)

def end_epoch_generate(epoch, _):
    print('\n Generating text after epoch: %d' % (epoch+1))
    texts_ex = ["From fairest creatures we desire increase,"]
    for text in texts_ex:
        sample = generate_next_emb(model_emb_m2m, text)
        print('%s' % (sample))

In [ ]:
model_emb_m2m = Sequential()
model_emb_m2m.add(...)
model_emb_m2m.add(LSTM(128, input_shape=(maximum_seq_length, 32), return_sequences=False))
model_emb_m2m.add(Dense(len(chars)))
model_emb_m2m.add(Activation('softmax'))

model_emb_m2m.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
model_emb_m2m.fit(X_emb, y_emb,
                  batch_size=128,
                  epochs=5,
                  validation_split = 0.2,
                  callbacks=[LambdaCallback(on_epoch_end=end_epoch_generate)])

In [ ]:
generate_next_emb(model_emb_m2m, "From fairest creatures we desire increase,")

In [ ]:
def get_tensor_emb(sentence, maximum_seq_length, voc):
    x = ...
    return x

In [ ]:
def generate_sample_emb(model, text, sample_function, *sampling_arg, num_generated=120, temperature=1.0):
    generated = text
    sentence = text[-maximum_seq_length:]
    for i in range(num_generated):
        x = ...
        predictions = ...
        next_index = sample_function(predictions, temperature, *sampling_arg)
        next_char =  ...
        generated += next_char
        sentence = sentence[1:] + next_char
    return(generated)

In [ ]:
generate_sample_emb(model_emb_m2m, text_ex.lower(), sample_top_k, 5, temperature = 0.7)